In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import os
from tqdm.auto import tqdm
from glob import glob
import cv2
import numpy as np
import pandas as pd
import PIL 
import urllib
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from random import uniform
from imgaug import augmenters as iaa
import torch.utils.data as td
import torchvision as tv
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

In [3]:
os.chdir('./drive/MyDrive/hack/')

In [ ]:
%%time
!unzip "./data.zip" -d "/content/drive/MyDrive/hack/data/"

In [4]:
transform = transforms.Compose([
                                
                                transforms.ColorJitter(brightness=.3),
                                transforms.RandomRotation(30),
                                transforms.RandomVerticalFlip(0.3),
                                transforms.RandomHorizontalFlip(0.3),
                                transforms.CenterCrop((256,256)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485 , 0.456, 0.406], [0.229, 0.224 , 0.225]),                              
                               ])


valid_transforms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485 , 0.456, 0.406], [0.229, 0.224 , 0.225]), 
    ])

In [5]:
train_dataset=datasets.ImageFolder('./data/train/',transform=transform)
valid_dataset=datasets.ImageFolder('./data/valid/',transform=valid_transforms)

In [6]:
# prepare data loaders (combine dataset and sampler)
batch_size=16
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [7]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

'cuda'

In [8]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=b1235e3f7f37887034d41571f225e33ce18ef107bd73d954dbc714e6f0399c01
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [9]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=5)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


In [10]:
model.load_state_dict(torch.load('hackaton1.pt'))

<All keys matched successfully>

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min')

In [ ]:
# number of epochs to train the model
n_epochs = 200

valid_loss_min = np.Inf # track change in validation loss

# keep track of training and validation loss
train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

train_acc = torch.zeros(n_epochs)
valid_acc = torch.zeros(n_epochs)
model.to(device)

for e in range(0, n_epochs):

   
    ###################
    # train the model #
    ###################
    model.train()
    for data, labels in tqdm(train_dataloader):
        # move tensors to GPU if CUDA is available
        data, labels = data.to(device), labels.to(device)
        # clear the gradients of all optimized variables
        
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        logits = model(data)
        # calculate the batch loss
        loss = criterion(logits, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss[e] += loss.item()
        
        ps = F.softmax(logits, dim=1)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.reshape(top_class.shape)
        train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
    
    train_loss[e] /= len(train_dataloader)
    train_acc[e] /= len(train_dataloader)
        
        
    ######################    
    # validate the model #
    ######################
    with torch.no_grad(): 
        model.eval()
        for data, labels in tqdm(valid_dataloader):
            # move tensors to GPU if CUDA is available
            data, labels = data.to(device), labels.to(device)
            # forward pass: compute predicted outputs by passing inputs to the model
            logits = model(data)
            # calculate the batch loss
            loss = criterion(logits, labels)
            # update average validation loss 
            valid_loss[e] += loss.item()
            
            ps = F.softmax(logits, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.reshape(top_class.shape)
            valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
            
    
    # calculate average losses
    valid_loss[e] /= len(valid_dataloader)
    valid_acc[e] /= len(valid_dataloader)
    
    scheduler.step(valid_loss[e])    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss[e], valid_loss[e]))
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
        e, train_acc[e], valid_acc[e]))
    
    # save model if validation loss has decreased
    if valid_loss[e] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss[e]))
        torch.save(model.state_dict(), 'hackaton1.pt')
        valid_loss_min = valid_loss[e]

  0%|          | 0/892 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch: 0 	Training Loss: 0.014423 	Validation Loss: 0.023727
Epoch: 0 	Training accuracy: 0.996356 	Validation accuracy: 0.995117
Validation loss decreased (inf --> 0.023727).  Saving model ...


  0%|          | 0/892 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

...

Epoch: 36 	Training Loss: 0.000242 	Validation Loss: 0.029487
Epoch: 36 	Training accuracy: 0.999930 	Validation accuracy: 0.995117


  0%|          | 0/892 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch: 37 	Training Loss: 0.000130 	Validation Loss: 0.029390
Epoch: 37 	Training accuracy: 1.000000 	Validation accuracy: 0.995117


  0%|          | 0/892 [00:00<?, ?it/s]

In [ ]:
plt.plot(train_loss)
plt.plot(valid_loss)

In [12]:
test_transform = transforms.Compose([transforms.Resize((256,256)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485 , 0.456, 0.406], [0.229, 0.224 , 0.225])
                                    ])

In [13]:
test_dataset=datasets.ImageFolder('./test/',transform=test_transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False)

In [14]:
acc=0
model.to(device)
with torch.no_grad(): 
    model.eval()
    for data, labels in tqdm(test_dataloader):
        # move tensors to GPU if CUDA is available
        data, labels = data.to(device), labels.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        logits = model(data)
        ps = F.softmax(logits, dim=1)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.reshape(top_class.shape)
        acc += torch.mean(equals.type(torch.float)).detach().cpu()
        
# calculate average losses
acc /= len(test_dataloader)
print('리멤버 team test accuracy: {:.6f}'.format(acc))

  0%|          | 0/89 [00:00<?, ?it/s]

리멤버 team test accuracy: 0.977528
